In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import os
import glob

In [2]:
parent_folder = '/content/HTML_FOLDER' #This is where the HTML Files are stored in a folder
output_csv = 'Google_search_result.csv' #Name the CSV file

In [3]:
def extract_links_titles_domains(parent_folder, output_csv, limit = None):
    all_links = []
    all_titles = []
    all_domains = []
    file_name = []
    total_entries = 0

    for html_file in sorted(os.listdir(parent_folder)):
        if html_file.endswith('.html'):
            with open(os.path.join(parent_folder, html_file), 'r', encoding='utf-8') as file:
                content = file.read()

            soup = BeautifulSoup(content, 'html.parser')

            # All links fall under the 'a' class
            # Extract links only that have an h3
            #This removes the links that are not search links but may appear as bubbles on google
            links_h3 = []

            for link in soup.find_all('a', href=True):
                href = link['href']
                if href.startswith('http'):
                    if link.find('h3'):
                        links_h3.append(link['href'])

            # Extract titles
            titles = []
            for title in soup.find_all('h3'):
                titles.append(title.get_text())

            # Extract domains
            domains = []
            for link in links_h3:
                domains.append(urlparse(link).netloc)

            #Make all lists the same length by filling in missing values with "NA"
            max_length = max(len(links_h3), len(titles), len(domains))

            #Fill shorter lists with "NA"
            links_h3 += ['NA'] * (max_length - len(links_h3))
            titles += ['NA'] * (max_length - len(titles))
            domains += ['NA'] * (max_length - len(domains))

            #Add a max limit
            for i in range(max_length):
              if limit and total_entries >= limit:
                break

              # Append the extracted data to the overall lists
              all_links.append(links_h3[i])
              all_titles.append(titles[i])
              all_domains.append(domains[i])
              file_name.append(html_file)
              total_entries = total_entries + 1

            if limit and total_entries >= limit:
              break



    #Create a DataFrame
    df = pd.DataFrame({
       'Links': all_links,
       'Titles': all_titles,
       'Domains': all_domains,
       'FileName': file_name
    })

    # Save to CSV or display the DataFrame
    df.to_csv(output_csv, index=False)
    print(f"Data saved to {output_csv}")
    print(df.head())



In [ ]:
#Arguments (3) are
#Parent Folder
#Output CSV file
#How many links to extract
extract_links_titles_domains(parent_folder, output_csv, limit = 60)
